In [ ]:
# origin dataset https://www.kaggle.com/a2015003713/militaryaircraftdetectiondataset
import os
import data_processor as dp
import numpy as np
# from PIL import Image
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l2, l1

In [ ]:
root = "input/archive_jets/dataset"
dataset = dp.TaskSpecificDataset(root)

In [ ]:
%%time
pandas_dataset = dp.PandasDataset(dataset.get_features(), root)
df = pandas_dataset.get_data_frame()
df.head()

In [ ]:
target_labels = df["class"].unique()
le = preprocessing.LabelEncoder()
le.fit(target_labels)

df["class"] = df["class"].apply(lambda x: le.transform([x])[0])
df

In [ ]:
%%time
# TODO change array subset to full array size
print("Dataset size: " + str(len(df["filename"])))
images_as_bytes = dp.ImageDataset(root, df["filename"][:100]).get_images()

# len(images_as_bytes)
print(images_as_bytes.shape)

for idx in range(0,10):
    plt.imshow(images_as_bytes[idx], interpolation='nearest')
    plt.show()

In [ ]:
# TODO change array subset to full array size
X = images_as_bytes
y = df["class"][:100]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [ ]:
# TODO have to skip it as issue with shpae (100,) instead of (100, 224, 224, 3) is still remain
scalar = MinMaxScaler()
# scalar.fit(x_train.reshape(x_train.shape[0],-1))
# x_train = scalar.transform(x_train.reshape(x_train.shape[0], -1)).reshape(x_train.shape)
# x_test = scalar.transform(x_test.reshape(x_test.shape[0], -1)).reshape(x_test.shape)
# scalar.fit(x_train.reshape(x_train.shape[0],-1))
# x_train = scalar.transform(x_train.reshape(x_train.shape[0],-1)).reshape(x_train.shape)
# x_test = scalar.transform(x_test.reshape(x_test.shape[0], -1)).reshape(x_test.shape)
x_train[0].shape[0]

In [ ]:
def create_cnn(data_shape_width, data_shape_height, data_shape_channel):

    kernel_size = 3
    
    model = Sequential()

    model.add(Conv2D(16, (kernel_size), strides=(1,1), padding='valid',input_shape=(data_shape_width, data_shape_height, data_shape_channel)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (5, 5), activation="relu", input_shape=(32, 32, 3)))

    model.add(Conv2D(32, (kernel_size), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(64, (kernel_size), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(64, (kernel_size), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2)))

    model.add(Conv2D(64, (kernel_size), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    
    model.add(Flatten())
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))

    return model

model = create_cnn(x_train[0].shape[0], x_train[0].shape[1], x_train[0].shape[2])

In [ ]:
# resize image close to the target resolution
# do padding to meet with the target resolution 

In [ ]:
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))

    return lrate

opt = Adam(lr=0.0001)
lrate = LearningRateScheduler(step_decay)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, 
                    batch_size=64, 
                    epochs=50, 
                    shuffle=True, 
                    verbose=2,
                    validation_data=(x_test, y_test), 
                    callbacks=[lrate])